# Interacting with your MISP instance

## Initializing the variables

We need to set a few variables:
- The URL of the MISP instance
- Your authentication key
- The certificate verification bool variable

In [ ]:
# The URL of the MISP instance to connect to
misp_url = 'https://training6.misp-community.org'

# The authentication key (can be found in the MISP
# web interface under _misp_url_/users/view/me -> Authkey)
misp_key = '_YOUR_MISP_AUTHENTICATION_KEY_'

# Should PyMISP verify the MISP certificate
misp_verifycert = False

Now we can use the API class

In [ ]:
from pymisp import PyMISP

misp = PyMISP(misp_url, misp_key, misp_verifycert)

# In order to return PyMISP objects whenever possible
misp.toggle_global_pythonify()

## Creating an Event

### Directly

In [ ]:
event1 = misp.add_event(
    {
        'info': 'Event from notebook',
        'distribution': 1,
        'analysis': 1,
        'threat_level_id': 1
    }
)

print(f'Event id of the created event: {event1.id}')

### Using the MISPEvent constructor

In [ ]:
from pymisp import MISPEvent

event_object = MISPEvent()
event_object.info = 'Event from noteboook 2'
event_object.distribution = 1
event_object.threat_level_id = 1
event_object.analysis = 1

event2 = misp.add_event(event_object)
print(f'Event id of the created event: {event2.id}')

## Updating an existing Event

### Fetching the Event

In [ ]:
# Pick an Event id
event_id = 10

event = misp.get_event(event_id)
print(event)

### Add an Attribute to the Event

In [ ]:
from pymisp import MISPAttribute

attribute_type = 'ip-src'
category = 'Network activity'

attribute = MISPAttribute()
attribute.type = attribute_type
attribute.value = '8.8.8.8'
attribute.to_ids = True
attribute.category = category

added_attribute = misp.add_attribute(event_id, attribute)
print(added_attribute.to_json(indent=4))

### Propose a new Attribute

In [ ]:
attribute = MISPAttribute()
attribute.type = attribute_type
attribute.value = '8.8.4.4'
attribute.to_ids = False
attribute.category = category

proposal = misp.add_attribute_proposal(event_id, attribute)
print(proposal.to_json(indent=4))

### Other operations on proposals

In [ ]:
# Get all the proposals for this event
proposals = misp.attribute_proposals(event_id)

# Get a specific proposal
proposal = misp.get_attribute_proposal()
print(proposal.to_json(indent=4))

# Accept a proposal
proposal = misp.accept_attribute_proposal()
print(proposal)

# Reject a proposal
proposal = misp.discard_attribute_proposal()
print(proposal)

### Propose changes to an Attribute

In [ ]:
from pymisp import MISPShadowAttribute

proposal = MISPShadowAttribute()
proposal.type = 'ip-dst'
proposal.category = 'External analysis'

attribute = misp.update_attribute_proposal(added_attribute.id, proposal)
print(attribute.to_json(indent=4))

attribute = misp.update_attribute_proposal(
    added_attribute.id,
    {
        'to_ids': False,
        'comment': 'This is crap'
    }
)
print(attribute.to_json(indent=4))

### Update the Event

We previously pushed some changes directly using the `PyMISP` constructor.

We can also get an event, update it locally and push the changes afterwards.

In [ ]:
from pymisp import MISPObject

event = misp.get_event(event_id)

attribute = {
    'type': 'ip-src',
    'value': '20.8.8.8',
    'to_ids': False,
    'category': 'Network activity'
}
misp_attribute = MISPAttribute()
misp_attribute.from_dict(**attribute)

event.add_attribute(**misp_attribute)
event.add_attribute('domain', 'circl.lu', disable_correlation=True, to_ids=False)

misp_object = MISPObject('file')
misp_object.add_attribute('filename', 'filename.exe')
misp_object.add_attribute('size-in-bytes', 1234, disable_correlation=True)
event.add_object(misp_object)

updated_event = misp.update_event(event)
print(updated_event.to_json(indent=4))

## Other available operations

Here are some other useful operations available with the `PyMISP` constructor

### Direct call (no validation)

Since most of the operations are also available with direct calls:

In [ ]:
misp.direct_call(f'attributes/add/{event_id}', {'type': 'ip-dst', 'value': '10.9.8.7'})

In [ ]:
misp.direct_call('events')

### Admin stuff

/!\ You need to have the permission to access those features /!\

#### Users

In [ ]:
misp.users()

#### Organisations

In [ ]:
misp.organisations()

#### Roles

In [ ]:
misp.roles()